# CBP Southwest Land Border Encounters
**Data Source:** [CBP Southwest Land Border Encounters](https://www.cbp.gov/newsroom/stats/southwest-land-border-encounters) <br>
**Download the Output:** [here](../data/extracted_data/cbp-apprehensions)


## Overview 

This notebook provides functionality to "scrape" or extract all data from the Tableau dashboards on the **CBP Southwest Land Border Encounters** page. 

This page contains two dashboards:
* Dashboard 1 contains information on FY Southwest Land Border Encounters 
* Dashboard 2 contains information on Us Border Patrol and Office of Field Operations Encounters. 


We will show how to scrape both dashboards. 
**Dashboard 1**
<img src="../misc/images/cbp_encounters_dash1.png" width=500 height=400 />
**Dashboard 2**
<img src="../misc/images/cbp_encounters_dash2.png" width=500 height=400 />


CBP does not provide this data in a spreadsheet format nor does it enable download of the data through the embedded Tableau chart. Therefore this code was developed to pull down all the data included every data point that exists with every possible combination of filters. 

**Please note:** When using the **data outputs** of this notebook for analysis or exploration that you will likely need to filter the data based available columns. The columns are based on the filter options in the tableau dashboards.

## Technical Approach

We use Python along with the [TableauScraper](https://github.com/bertrandmartel/tableau-scraping) Python library to extract all data from the Tableau dashboards. These dashboards have many filters that provide different slices of data. We will use code to request all possible slices and then combine that data into a single dataframe (or table) and save it out to csv. 

## Skills Learned
* How to download data from Tableau dashboard that doesn't not allow you to directly download data. 

## The Code 

**PLEASE NOTE**: We have made this notebook READ only to ensure you receive all updates we make to it. Do not edit this notebook directly, create a copy instead.

To customize and experiment with this notebook:
1. Create a copy: `Select File -> Make a Copy` at the top-left of the notebook
2. Unlock cells in your copy: Press `CMD + A` on your keyboard to select all cells, then click the small unlocked padlock button near the mid-top right of the notebook.


In [4]:
# Import helper python libraries
from tableauscraper import TableauScraper as TS
import itertools
import logging
import time
import pprint
import pandas as pd

import requests
from bs4 import BeautifulSoup
import urllib
from urllib.parse import unquote


pp = pprint.PrettyPrinter(indent=4)
import logging.config

logging.config.dictConfig(
    {
        "version": 1,
        "disable_existing_loggers": True,
    }
)

------------
------------

## 1. Getting the Dashboard URLs

The Tableau dashboards on the CBP page are embedded in the CBP web page. The dashboards themselves are hosted by Tableau, so we need to get the actual link to the dashboard, not the link to the website where they are embedded. 
 (This is sort of confusing) 

### 1a. With Code

Below we use some code to extract the URLs that are for dashboards on the site. We use a web scraping library called Beautiful Soup to extract information from the web page. 

In [5]:
def get_tableau_dashboard_url_from_tableau_placeholder(item):
    """
    Takes an item with a class of tableauPlaceholder and returns a valid url for a dashboard

    Parameters:
        item: Beautiful soup item with class of tableauPlaceholder

    Returns:
        A valid url for a Tableau dashboard
    """
    url = unquote(item.find("param", {"name": "host_url"})["value"]).strip("/")
    url += (
        "/"
        + unquote(item.find("param", {"name": "site_root"})["value"]).strip("/")
        + "/views"
    )
    url += "/" + unquote(item.find("param", {"name": "name"})["value"]).strip("/")
    return url


# Below we take the URL of the CPB webpage, and search for tableaPlaceholder elements in the
# webpage background information.
url = "https://www.cbp.gov/newsroom/stats/southwest-land-border-encounters"
reqs = requests.get(url)  # get the URL's data
soup = BeautifulSoup(reqs.text, "html.parser")  # parse it with Beautifulsoup

# Find the tableauPlaceholder divs and get the dashboard URLs
for idx, item in enumerate(soup.find_all("div", {"class": "tableauPlaceholder"})):
    print(idx + 1, get_tableau_dashboard_url_from_tableau_placeholder(item))

1 https://publicstats.cbp.gov/t/PublicFacing/views/CBPSBOEnforcementActionsDashboardsOCTFY22/SBOEncounters10935
2 https://publicstats.cbp.gov/t/PublicFacing/views/CBPSBOEnforcementActionsDashboardsOCTFY22/SBObyMonthDemo10935


And there we go - we have two links for the two dashboards on the CBP page. 

Now there is another way to find these links that will work more consistently across different websites, so we will also show how to find the links manually with developer tools. 

----------

### 1b. With Developer Tools


To do this you must access the Developer Tools in your web browser. For Google Chrome you can click on 

**View --> Developer --> Developer Tools**

See the bottom of the image below to see what the developer tools may look like. Note that the display may open to the right of your screen or the bottom. The developer tools let us look at the source information of the page. Web page source info provides the instructions/information a browser needs to display the web page. Buried in that information is the actual true link of the Tableau dashboards.  

Let's get those links

<img src="../misc/images/chrome_dev_tools1.png"  />

Once you have **devtools** open you can click on the **Elements** button and then the small arrow in a box. 

<img src="../misc/images/chrome_devtools2.png"  />

Once selected **click** near the top of the Tableau dashboard. This will highlight the dashboard element in the page source. 

<img src="../misc/images/chrome_devtools_3.png"  />

<br>
In the dev tools window and scroll up (or possibly down) and look for a URL. 

**See below - the part that says `https://publicstats....` this is the actual URL to the first dashboard**

<img src="../misc/images/chrome_devtools_4_url.png"  />

Copy everything up to `isGuestRedirectFromVizportal` - then you have your URL, you can repeat for the second URL 

----------------------------
------------------------

## 2. Scraping Dashboards

### 2a. Get data from Dashboard 1

In [6]:
dashboard1_url = "https://publicstats.cbp.gov/t/PublicFacing/views/CBPSBOEnforcementActionsDashboardsOCTFY22/SBOEncounters10935"

Here we activate (instantiate is the technical term) the TableauScraper library and then load data from the dashboard URL. 

In [131]:
# Create a tableau scraper object
ts = TS()

# Pass the CBP dashboard URL to the tableau scraper object, which will then grab the data from the dashboard
ts.loads(dashboard1_url)

**NOTE**: Comments below refer to the first dashboard primarily, but all this code can be applied to the second dashboard as well. 


Tableau dashboards are generally made up of multiple visualizations all combined into a single dashboard. For the first CBP dashboard we have drop down filters at the top, then a line chart and then a table. Behind the scenes there may be even granularity such as a specific visualization just for the totals column etc. 

For our purposes we need to identify 2 main things ... 
1. What part of the visualization do we want to extract data from 
2. Which part of the visualization are the filter drop downs linked to. 

The first part is pretty straightforward, for the first dashboard we want to extract the table data (though the line data is basically the same as well , just seen as a line chart). 


Let's us the `ts` object we created above and see what different components make up the first dashboard. 

In [132]:
def print_worksheets(ts):
    for t in ts.getWorkbook().worksheets:
        print(t.name)


print_worksheets(ts)

SBO FYTD Comparison
SBO Line Graph
SBO Reset
SBO Table
SBO Total Only


So we have 5 different parts, (so confusing ! ). But Looking at these items we can guess that the `SBO Table` is the primary table and that the `SBO Line Graph` is the primary line chart. The rest are totals, and other smaller components. 

So going back to finding the filters..  The CBP tableau dashboards have different filters available for the user to apply, the filters linked to one of these specific visualization elements. 

Let's apply some code to see which of the elements has filters attached to it, thankfully **the filter will apply to all elements** so if even if it lives on a different element than we want it will likely apply to the element we do want.

In [133]:
def find_filters_worksheet(ts):

    """
    Function to search the dashboard to find the worksheet that manages
    the filters that are avaiable on the dashboard.
    """
    workbook = ts.getWorkbook()
    filters_ws = None
    wb_names = []
    for t in workbook.worksheets:
        filters = t.getFilters()
        if len(filters) > 0:
            print("-" * 90)
            print(
                f"Filters Presesnt on element name --> {t.name}"
            )  # show worksheet name
            pp.pprint(filters)  # show dataframe for this worksheet
            filters_ws = t.name
            print("-" * 90)
        else:
            print("\nno filters found on element --> ", t.name)

        wb_names.append(t.name)
    return filters_ws, wb_names

**So now let's search the dashboard for filters**

In [134]:
filters_ws, wb_names = find_filters_worksheet(ts)


no filters found on element -->  SBO FYTD Comparison
------------------------------------------------------------------------------------------
Filters Presesnt on element name --> SBO Line Graph
[   {   'column': 'Citizenship Grouping',
        'globalFieldName': '[federated.1xhccc00nlacbx14ajs101w1uee1].[none:Citizenship '
                           'Grouping:nk]',
        'ordinal': 0,
        'selection': [   'El Salvador',
                         'Guatemala',
                         'Honduras',
                         'Mexico',
                         'Other',
                         'all'],
        'selectionAlt': [   {   'columnFullNames': ['[Citizenship Grouping]'],
                                'domainTables': [   {   'isSelected': True,
                                                        'label': 'El '
                                                                 'Salvador'}],
                                'fn': '[federated.1xhccc00nlacbx14ajs101w1uee1].[none

Above we can see which element has the filters, using the first dashboard the filters are on the `SBO Line Graph` element (if using the second URL the filters are on the `All MoM Change Podium` element).

 The `filters_ws` holds the value for which worksheet has the filters and saves it automatically. 

In [135]:
print(filters_ws)

SBO Line Graph


#### Extract the Data

So now that we know which part of the dashboard we want to extract data from and which part has the filters information we can extract all the data with all the different filter combinations automatically into one spreadsheet. 

Imagine how long this would take to do manually :) 

In [136]:
def unpack_filter_information(ts, filters_ws, skip_filter=[]):

    """
    Function process filters and return object of information on those filters,
    on all the combinations, columns names, and other useful information.

    skip_filter: is a list with column names that should be skipped, you are likley
    skipping columns because the you do not need to apply that filter.

    returns dictionary
    """

    # Specifically ask for the Line Graph since it has the filters
    ws = ts.getWorksheet(filters_ws)

    # Get the different filter names and their corresponding options, then store this data in a dictionary,
    # where the filter names are keys and the values are lists of corresponding filter options
    filter_master_list = ws.getFilters()
    filter_possible_values_list = {i["column"]: i["values"] for i in filter_master_list}
    print("\nFilters and their possible values:")
    pp.pprint(filter_possible_values_list)

    # Grab just the column names
    all_filter_columns = [i["column"] for i in filter_master_list]
    # Drop the fiscal year because the chart automatically includes all fiscal years
    for col in skip_filter:
        try:
            all_filter_columns.remove(col)
        except ValueError:
            print(f"{col} not present")

    # The code beloew creates an exhaustive
    # list of all possible filter combinations, this does not control for
    # combinations that don't exist though, meaning that some filter comnbinations
    # that don't have any valid data may be attempted, this is fine though we just
    # won't get data back for those combinations.
    all_f_values = []
    for f in filter_master_list:
        if f["column"] not in skip_filter:
            vals = f["values"]
            all_f_values.append([None] + vals)
    all_filter_combinations = list(itertools.product(*all_f_values))

    filter_data = {
        "master_list": filter_master_list,
        "filter_columns": all_filter_columns,
        "filter_combinations": all_filter_combinations,
    }
    print("\nFilter on:")
    pp.pprint(filter_data["filter_columns"])
    return filter_data

Now we create an exhaustive list of Dashboard 1 filter combinations - Note we skip `Fiscal Year` because it is broken out in the chart already. We found the exact name of this filter above where we called the `find_filters_worksheet` function


In [137]:
filter_data = unpack_filter_information(
    ts,
    filters_ws,
    skip_filter=["Fiscal Year"],
)


Filters and their possible values:
{   'Citizenship Grouping': [   'El Salvador',
                                'Guatemala',
                                'Honduras',
                                'Mexico',
                                'Other'],
    'Component': ['Office of Field Operations', 'U.S. Border Patrol'],
    'Demographic': [   'Accompanied Minors',
                       'FMUA',
                       'Single Adults',
                       'UC / Single Minors'],
    'Fiscal Year': ['2019', '2020', '2021', '2022 (FYTD)'],
    'Title of Authority': ['Title 8', 'Title 42']}

Filter on:
['Citizenship Grouping', 'Component', 'Demographic', 'Title of Authority']


**See the various filter combinations**

Above we see the **Filter on:...**, those columns correspond to the values we see below.

When a value is `None` that means the filter is not applied so it is equivalent to `all`


In [138]:
pp.pprint(filter_data["filter_combinations"])

[   (None, None, None, None),
    (None, None, None, 'Title 8'),
    (None, None, None, 'Title 42'),
    (None, None, 'Accompanied Minors', None),
    (None, None, 'Accompanied Minors', 'Title 8'),
    (None, None, 'Accompanied Minors', 'Title 42'),
    (None, None, 'FMUA', None),
    (None, None, 'FMUA', 'Title 8'),
    (None, None, 'FMUA', 'Title 42'),
    (None, None, 'Single Adults', None),
    (None, None, 'Single Adults', 'Title 8'),
    (None, None, 'Single Adults', 'Title 42'),
    (None, None, 'UC / Single Minors', None),
    (None, None, 'UC / Single Minors', 'Title 8'),
    (None, None, 'UC / Single Minors', 'Title 42'),
    (None, 'Office of Field Operations', None, None),
    (None, 'Office of Field Operations', None, 'Title 8'),
    (None, 'Office of Field Operations', None, 'Title 42'),
    (None, 'Office of Field Operations', 'Accompanied Minors', None),
    (None, 'Office of Field Operations', 'Accompanied Minors', 'Title 8'),
    (None, 'Office of Field Operations', '

### Now let's pull down the data

**Data Extraction Function**

We will create a function to pull down the data and parameterize some of the arguments 

In [139]:
def get_dashboard_data(
    url: str,
    all_filter_columns: list,
    all_filter_combinations,
    filter_worksheet,
    data_worksheet,
):
    """
    Function to extract data from a Tableau Dashboard

    Parameters:
        url: URL of dashboard
        all_filter_columns: list of columns we will use for filtering
        all_filter_combinations: list of tuples containing all filter combinations
        filter_worksheet: worksheet where filters exist
        data_worksheet: worksheet we want to extract data from

    return dashboard data extracted from tableau dashboard element
    """
    failed_combination = []
    tableau_dataframe = pd.DataFrame()
    for filter_combination in all_filter_combinations:
        print("Attempting Filter Combination", filter_combination)
        ts = TS()
        ts.loads(url)
        workbook = ts  # in case all filters are null
        worksheet = ts.getWorksheet(filter_worksheet)
        try:
            for idx, col in enumerate(all_filter_columns):
                # If it is none it means we are not applying any filter option for the dropdown filter
                if filter_combination[idx] is None:
                    continue
                else:
                    # apply the individual filter and continue iterating
                    worksheet = workbook.getWorksheet(filter_worksheet)
                    workbook = worksheet.setFilter(
                        col, filter_combination[idx], filterDelta=True
                    )
            subset_worksheet = workbook.getWorksheet(data_worksheet)
            subset_data = subset_worksheet.data
            if len(subset_data) > 0:  # Only do this if we have data
                # Now we iterate over the filter and label the data with
                # what filters were applied.
                for col, val in list(zip(all_filter_columns, filter_combination)):
                    if val is None:
                        val = "all"
                    subset_data.loc[:, col] = val

                # append the data to our master dataframe
                tableau_dataframe = tableau_dataframe.append(subset_data)
            else:
                print(f"WARNING No Length on {filter_combination}")
                failed_combination.append(filter_combination)
        except Exception as e:
            print(f"WARNING on {filter_combination} \n {e}")
            failed_combination.append(filter_combination)
    return tableau_dataframe, failed_combination

Now we will run it ... This may take about 20 minutes

In [140]:
dataset1, failed_combination1 = get_dashboard_data(
    dashboard1_url,
    filter_data["filter_columns"],
    filter_data["filter_combinations"],
    filters_ws,
    "SBO Table",
)

Attempting Fitler Combination (None, None, None, None)
Attempting Fitler Combination (None, None, None, 'Title 8')
Attempting Fitler Combination (None, None, None, 'Title 42')
Attempting Fitler Combination (None, None, 'Accompanied Minors', None)
Attempting Fitler Combination (None, None, 'Accompanied Minors', 'Title 8')
Attempting Fitler Combination (None, None, 'Accompanied Minors', 'Title 42')
Attempting Fitler Combination (None, None, 'FMUA', None)
Attempting Fitler Combination (None, None, 'FMUA', 'Title 8')
Attempting Fitler Combination (None, None, 'FMUA', 'Title 42')
Attempting Fitler Combination (None, None, 'Single Adults', None)
Attempting Fitler Combination (None, None, 'Single Adults', 'Title 8')
Attempting Fitler Combination (None, None, 'Single Adults', 'Title 42')
Attempting Fitler Combination (None, None, 'UC / Single Minors', None)
Attempting Fitler Combination (None, None, 'UC / Single Minors', 'Title 8')
Attempting Fitler Combination (None, None, 'UC / Single Minors

Attempting Fitler Combination ('El Salvador', 'U.S. Border Patrol', 'Single Adults', 'Title 42')
Attempting Fitler Combination ('El Salvador', 'U.S. Border Patrol', 'UC / Single Minors', None)
Attempting Fitler Combination ('El Salvador', 'U.S. Border Patrol', 'UC / Single Minors', 'Title 8')
Attempting Fitler Combination ('El Salvador', 'U.S. Border Patrol', 'UC / Single Minors', 'Title 42')
Attempting Fitler Combination ('Guatemala', None, None, None)
Attempting Fitler Combination ('Guatemala', None, None, 'Title 8')
Attempting Fitler Combination ('Guatemala', None, None, 'Title 42')
Attempting Fitler Combination ('Guatemala', None, 'Accompanied Minors', None)
Attempting Fitler Combination ('Guatemala', None, 'Accompanied Minors', 'Title 8')
Attempting Fitler Combination ('Guatemala', None, 'Accompanied Minors', 'Title 42')
Attempting Fitler Combination ('Guatemala', None, 'FMUA', None)
Attempting Fitler Combination ('Guatemala', None, 'FMUA', 'Title 8')
Attempting Fitler Combination

WARNING on ('Honduras', 'U.S. Border Patrol', 'Accompanied Minors', 'Title 42') 
 'NoneType' object is not subscriptable
Attempting Fitler Combination ('Honduras', 'U.S. Border Patrol', 'FMUA', None)
Attempting Fitler Combination ('Honduras', 'U.S. Border Patrol', 'FMUA', 'Title 8')
Attempting Fitler Combination ('Honduras', 'U.S. Border Patrol', 'FMUA', 'Title 42')
Attempting Fitler Combination ('Honduras', 'U.S. Border Patrol', 'Single Adults', None)
Attempting Fitler Combination ('Honduras', 'U.S. Border Patrol', 'Single Adults', 'Title 8')
Attempting Fitler Combination ('Honduras', 'U.S. Border Patrol', 'Single Adults', 'Title 42')
Attempting Fitler Combination ('Honduras', 'U.S. Border Patrol', 'UC / Single Minors', None)
Attempting Fitler Combination ('Honduras', 'U.S. Border Patrol', 'UC / Single Minors', 'Title 8')
Attempting Fitler Combination ('Honduras', 'U.S. Border Patrol', 'UC / Single Minors', 'Title 42')
Attempting Fitler Combination ('Mexico', None, None, None)
Attempt

WARNING on ('Other', 'U.S. Border Patrol', 'Accompanied Minors', 'Title 8') 
 'NoneType' object is not subscriptable
Attempting Fitler Combination ('Other', 'U.S. Border Patrol', 'Accompanied Minors', 'Title 42')
WARNING on ('Other', 'U.S. Border Patrol', 'Accompanied Minors', 'Title 42') 
 'NoneType' object is not subscriptable
Attempting Fitler Combination ('Other', 'U.S. Border Patrol', 'FMUA', None)
Attempting Fitler Combination ('Other', 'U.S. Border Patrol', 'FMUA', 'Title 8')
Attempting Fitler Combination ('Other', 'U.S. Border Patrol', 'FMUA', 'Title 42')
Attempting Fitler Combination ('Other', 'U.S. Border Patrol', 'Single Adults', None)
Attempting Fitler Combination ('Other', 'U.S. Border Patrol', 'Single Adults', 'Title 8')
Attempting Fitler Combination ('Other', 'U.S. Border Patrol', 'Single Adults', 'Title 42')
Attempting Fitler Combination ('Other', 'U.S. Border Patrol', 'UC / Single Minors', None)
Attempting Fitler Combination ('Other', 'U.S. Border Patrol', 'UC / Single

**Check if anything failed**

In [141]:
print(len(failed_combination1))
pp.pprint(failed_combination1)

24
[   (None, 'Office of Field Operations', 'FMUA', 'Title 42'),
    (None, 'U.S. Border Patrol', 'Accompanied Minors', None),
    (None, 'U.S. Border Patrol', 'Accompanied Minors', 'Title 8'),
    (None, 'U.S. Border Patrol', 'Accompanied Minors', 'Title 42'),
    ('El Salvador', 'Office of Field Operations', 'FMUA', 'Title 42'),
    ('El Salvador', 'U.S. Border Patrol', 'Accompanied Minors', None),
    ('El Salvador', 'U.S. Border Patrol', 'Accompanied Minors', 'Title 8'),
    ('El Salvador', 'U.S. Border Patrol', 'Accompanied Minors', 'Title 42'),
    ('Guatemala', 'Office of Field Operations', 'FMUA', 'Title 42'),
    ('Guatemala', 'U.S. Border Patrol', 'Accompanied Minors', None),
    ('Guatemala', 'U.S. Border Patrol', 'Accompanied Minors', 'Title 8'),
    ('Guatemala', 'U.S. Border Patrol', 'Accompanied Minors', 'Title 42'),
    ('Honduras', 'Office of Field Operations', 'FMUA', 'Title 42'),
    ('Honduras', 'U.S. Border Patrol', 'Accompanied Minors', None),
    ('Honduras', 'U.

Note if there are failure, some/all of these failures may be because the combination results in no data. You can verify this manually be attempting these combinations with the actual tableau dashboard. 

#### Now Same Thing but Just the code 

Now that we have walked through all the steps to collect the first dashboard, we just wanted to highlight how short the actual code to extract is once you know how this works, AND what elements you want to extract from the dashboard.

Below we just have all the steps to collect the first dashboard without all the comments 

In [142]:
ts = TS()
ts.loads(dashboard1_url)
data_element_target = "SBO Table"

filters_ws, wb_names = find_filters_worksheet(ts)

# Dashboard 1- Note we skip fiscal year because it is broken out in the chart already
filter_data = unpack_filter_information(
    ts,
    filters_ws,
    skip_filter=["Fiscal Year"],
)

dataset1, failed_combination1 = get_dashboard_data(
    dashboard1_url,
    filter_data["filter_columns"],
    filter_data["filter_combinations"],
    filters_ws,
    data_element_target,
)

print(len(failed_combination))
pp.pprint(failed_combination)

**Export the data to csv**

Below we just grab today's year, month and date and we will include this in the name of the output file. 


In [8]:
today_date = time.strftime("%Y-%m-%d")

Now run the export command

In [ ]:
dataset1.to_csv(
    f"../data/extracted_data/cbp-tableau/cbp-encounters-dashboard-1-{today_date}.csv"
)

-------------------------

### 2b. Get data from Dashboard 2

In [ ]:
dashboard2_url = "https://publicstats.cbp.gov/t/PublicFacing/views/CBPSBOEnforcementActionsDashboardsOCTFY22/SBObyMonthDemo10935"

In [143]:
ts = TS()
ts.loads(dashboard2_url)
print_worksheets(ts)

All MoM Change Podium
Demo FYTD by Month (2)
OFO MoM Change
Reset SBO by Month
USBP MoM Change


So I believe we want to collect the table data in the second dashboard, so I'm guessing that `Demo FYTD by Month (2)` is the correct element. This is a bit confusing but if you look at the dashboard and compare you can make an educated guess about which element holds the information you want. If you get it wrong you can always rerun the code with a different element. :) 

Let's see where the filters live in this dashboard.

In [144]:
filters_ws, wb_names = find_filters_worksheet(ts)

------------------------------------------------------------------------------------------
Filters Presesnt on element name --> All MoM Change Podium
[   {   'column': 'Citizenship Grouping',
        'globalFieldName': '[federated.1xhccc00nlacbx14ajs101w1uee1].[none:Citizenship '
                           'Grouping:nk]',
        'ordinal': 0,
        'selection': [   'El Salvador',
                         'Guatemala',
                         'Honduras',
                         'Mexico',
                         'Other',
                         'all'],
        'selectionAlt': [   {   'columnFullNames': ['[Citizenship Grouping]'],
                                'domainTables': [   {   'isSelected': True,
                                                        'label': 'El '
                                                                 'Salvador'}],
                                'fn': '[federated.1xhccc00nlacbx14ajs101w1uee1].[none:Citizenship '
                                

Ok looks like the filters are on the `All MoM Change Podium` element. 

In [145]:
print("filters_ws:", filters_ws)

filters_ws: All MoM Change Podium


In [146]:
# Dashboard 2  - note here we skip demographics becauase they are broken out in the chart already
filter_data = unpack_filter_information(
    ts,
    filters_ws,
    skip_filter=["Demographic"],
)


Filters and their possible values:
{   'Citizenship Grouping': [   'El Salvador',
                                'Guatemala',
                                'Honduras',
                                'Mexico',
                                'Other'],
    'Demographic': [   'Accompanied Minors',
                       'FMUA',
                       'Single Adults',
                       'UC / Single Minors'],
    'Title of Authority': ['Title 8', 'Title 42']}

Filter on:
['Citizenship Grouping', 'Title of Authority']


Now let's collect the dashboard data -  This takes about 5 min

In [147]:
dataset2, failed_combination2 = get_dashboard_data(
    dashboard2_url,
    filter_data["filter_columns"],
    filter_data["filter_combinations"],
    filters_ws,
    "Demo FYTD by Month (2)",
)

Attempting Fitler Combination (None, None)
Attempting Fitler Combination (None, 'Title 8')
Attempting Fitler Combination (None, 'Title 42')
Attempting Fitler Combination ('El Salvador', None)
Attempting Fitler Combination ('El Salvador', 'Title 8')
Attempting Fitler Combination ('El Salvador', 'Title 42')
Attempting Fitler Combination ('Guatemala', None)
Attempting Fitler Combination ('Guatemala', 'Title 8')
Attempting Fitler Combination ('Guatemala', 'Title 42')
Attempting Fitler Combination ('Honduras', None)
Attempting Fitler Combination ('Honduras', 'Title 8')
Attempting Fitler Combination ('Honduras', 'Title 42')
Attempting Fitler Combination ('Mexico', None)
Attempting Fitler Combination ('Mexico', 'Title 8')
Attempting Fitler Combination ('Mexico', 'Title 42')
Attempting Fitler Combination ('Other', None)
Attempting Fitler Combination ('Other', 'Title 8')
Attempting Fitler Combination ('Other', 'Title 42')


In [148]:
print(len(failed_combination2))
pp.pprint(failed_combination2)

0
[]


In [149]:
dataset2

,Component-value,Component-alias,Demographic-value,Demographic-alias,Month (abbv)-value,Month (abbv)-alias,SUM(Encounter Count)-alias,ATTR(Demographic (copy))-alias,Citizenship Grouping,Title of Authority
0,Office of Field Operations,Office of Field Operations,%all%,%all%,%all%,%all%,5728,%many-values%,all,all
1,Office of Field Operations,Office of Field Operations,%all%,%all%,OCT,OCT,5728,%many-values%,all,all
2,Office of Field Operations,Office of Field Operations,Accompanied Minors,AM,%all%,%all%,187,Accompanied Minors,all,all
3,Office of Field Operations,Office of Field Operations,Accompanied Minors,AM,OCT,OCT,187,Accompanied Minors,all,all
4,Office of Field Operations,Office of Field Operations,UC / Single Minors,UC / Single Minors,%all%,%all%,160,UC / Single Minors,all,all
...,...,...,...,...,...,...,...,...,...,...
9,U.S. Border Patrol,U.S. Border Patrol,FMUA,FMUA,OCT,OCT,550,FMUA,Other,Title 42
10,U.S. Border Patrol,U.S. Border Patrol,Single Adults,Single Adults,%all%,%all%,1228,Single Adults,Other,Title 42
11,U.S. Border Patrol,U.S. Border Patrol,Single Adults,Single Adults,OCT,OCT,1228,Single Adults,Other,Title 42
12,%all%,%all%,%all%,%all%,%all%,%all%,1817,%many-values%,Other,Title 42


Now run the export command

In [152]:
dataset2.to_csv(
    f"../data/extracted_data/cbp-tableau/cbp-encounters-dashboard-2-{today_date}.csv"
)

----------

# End